In [23]:
!pip install kaggle

In [24]:
import kaggle

In [25]:
#download dataset using kaggel API
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [26]:
#extract file from zipfile
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip') 
zip_ref.extractall() # extract file to dir
zip_ref.close() # close file



In [27]:
#read data from the file and handle null values
import pandas as pd
df = pd.read_csv('orders.csv',na_values=['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [28]:
#rename columns name to lowercase and add underscore sign
#df.columns=df.columns.str.lower()
#df.columns=df.columns.str.replace(' ','_')
df.head(5)

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [29]:
#derive new columns discount,profit,sales
#df['discount']=df['list_price']*df['discount_percent']/100
#df['sale_price']=df['list_price']-df['discount']
#df['profit']=df['sale_price']-df['cost_price']
df

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,2023-02-18,Second Class,Consumer,United States,Miami,Florida,33180,South,Furniture,Furnishings,FUR-FU-10001889,30,30,3,4
9990,9991,2023-03-17,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Furniture,Furnishings,FUR-FU-10000747,70,90,2,4
9991,9992,2022-08-07,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Technology,Phones,TEC-PH-10003645,220,260,2,2
9992,9993,2022-11-19,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Office Supplies,Paper,OFF-PA-10004041,30,30,4,3


In [30]:
#convert order_date from object data type to datetime
df['order_date']=pd.to_datetime(df.order_date)
df.dtypes


AttributeError: 'DataFrame' object has no attribute 'order_date'

In [ ]:
#drop cost_price,list_price and discount_percent columns
df.drop(columns=['cost_price','list_price','discount_percent'],inplace=True)
df

In [ ]:
!pip install psycopg2-binary

In [36]:
!pip install sqlalchemy

In [48]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Define connection parameters
hostname = 'localhost'
database = 'Data_analysis'
username = 'postgres'
pwd = 'admin123'
port_id = 5432

# Initialize connection and cursor variables
conn = None
curr = None

try:
    # Establish connection using psycopg2
    conn = psycopg2.connect(
        host=hostname,
        dbname=database,
        user=username,
        password=pwd,
        port=port_id
    )
    curr = conn.cursor()
    print("Connection successful")
    
    # Assuming df is your DataFrame
    # Create SQLAlchemy engine
    conn_string = f'postgresql+psycopg2://{username}:{pwd}@{hostname}:{port_id}/{database}'
    engine = create_engine(conn_string)

    

    df.to_sql('df_orders', con=engine, index=False, if_exists='append')
    print("Data loaded successfully")

except Exception as error:
    print("Error:", error)

finally:
    if curr is not None:
        curr.close()
    if conn is not None:
        conn.close()


Connection successful
Data loaded successfully
